In [1]:
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry.polygon import Polygon
import numpy as np
import pyproj
import math

In [2]:
# Read in the GeoJSON file
zone = gpd.read_file('map.geojson')

In [3]:
zone = zone.to_crs(epsg = 3857)
zone

,geometry
0,"POLYGON ((6521887.454 4638946.148, 6521887.454..."


In [4]:
xmin, ymin, xmax, ymax = zone.total_bounds
xmin, ymin, xmax, ymax

(6521887.453627451, 724394.7404732915, 11012147.2208873, 4638946.148399052)

In [5]:
# Set the resolution for the grid (10km x 10km)
res = 10000

In [6]:
cols = list(np.arange(xmin, xmax + res, res))
rows = list(np.arange(ymin, ymax + res, res))

In [7]:
print(len(rows))
len(cols)

393


451

In [8]:
polygons = []
for x in cols[:-1]:
    for y in rows[:-1]:
        polygons.append(Polygon([(x,y), (x+res, y), (x+res, y+res), (x, y+res)]))

In [9]:
grid_10km = gpd.GeoDataFrame({'geometry': polygons}, crs= 'EPSG:3857') 

In [10]:
# Add a buffer of 1km around each cell
grid_10km['geometry'] = grid_10km['geometry'].buffer(res/10)

In [12]:
# Add a grid ID column
grid_10km['grid_id'] = range(1, len(grid_10km) + 1)

In [13]:
# Change CRS because otherwise it returns an error in GEE
grid_10km = grid_10km.to_crs(epsg = 4326)

In [14]:
grid_10km.shape

(176400, 2)

In [15]:
grid_10km.to_file("grid_10km_new", driver="GeoJSON")

C:\Users\ayako\anaconda3\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [16]:
grid_10km.to_file("grid_10km_new", driver='ESRI Shapefile')

C:\Users\ayako\anaconda3\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
